# LangChain para INVIAS

## 🔒 1. Instalación de librerías

### 🔓  

Solo ejecutar este script si no se han instalado los paquetes para desarrollar el código

In [ ]:
""" 
Lista de los paquetes a instalar:

    upgrade:
        pip
        setuptools
        wheel
    Packages:
        langchain 
        pypdf 
        openai 
        chromadb 
        tiktoken
        langchain-community
"""
import subprocess

comandos = [
    ["pip", "install", "--upgrade", "pip", "setuptools", "wheel"],
    ["pip", "install", "langchain", "pypdf", "openai", "chromadb", "tiktoken"],
    ["pip", "install", "-U", "langchain-community"],
    ["python", "-m", "pip", "install", "--upgrade", "pip"]
]

log_path = "instalacion_log.txt"

with open(log_path, "w", encoding="utf-8") as log_file:
    for i, cmd in enumerate(comandos, start=1):
        log_file.write(f"\n🔧 Ejecutando comando {i}: {' '.join(cmd)}\n")
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        log_file.write("✅ STDOUT:\n")
        log_file.write(result.stdout + "\n")
        
        if result.stderr:
            log_file.write("⚠️ STDERR:\n")
            log_file.write(result.stderr + "\n")

print(f"✅ Resultado guardado en {log_path}")


## 2. Configuración de API Key de OpenAI

In [1]:
import os
from openai import OpenAI
# Recuperar la clave API de la variable de entorno
api_key_environ = os.environ.get("OPENAI_API_KEY")
 
# Verificar que la clave API esté disponible
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada o está vacía.")
 
# Inicializar el cliente de OpenAI con la clave API
client = OpenAI(api_key=api_key_environ)
 
# Usar el cliente para tus tareas
print("¡Cliente de OpenAI inicializado correctamente!")

¡Cliente de OpenAI inicializado correctamente!


## 🔒 3. Carga de documents

### 🔓  

Solo ejecutar este script si no se ha hecho el proceso de embedding

In [37]:
import requests
from langchain.document_loaders import PyPDFLoader
import os

relative_pdf_path = "../../../assets/DG_docs/PDFs_test/"

ml_papers = []

for i, file_name in enumerate(os.listdir(relative_pdf_path)):
    if file_name.lower().endswith(".pdf"):
        full_pdf_path = os.path.join(relative_pdf_path,file_name)
        print(f"📄 Cargando {file_name}")

        loader = PyPDFLoader(full_pdf_path)
        data = loader.load() # AI_Queries\code_explanation\ai_query-langc_v01-PyPDFLoader(filename).loader.load()_usage.md
        ml_papers.extend(data) # AI_Queries\code_explanation\ai_query-langc_v01-.extend_usage.md
        # print (ml_papers) # AI_Queries/code_explanation/ai_query-langc_v01-list_start_end_usage.md
# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Esto es todo el contenido de `ml_papers:`')
print(f"""
🆗 Todos los documentos estan cargados en ml_papers.
➖ Total de fragmentos: {len(ml_papers)}
➖ Los fragmentos son cada una de las hojas de cada uno de los {len(os.listdir(relative_pdf_path))} archivos en la carpeta {relative_pdf_path}
➖ Este script se ejecuta desde {os.getcwd()}
➖ Este es el contenido de la última hoja cargada {ml_papers[-1]}
""")

📄 Cargando 2024S-VBOG-054699.pdf
📄 Cargando 2024S-VBOG-056838.pdf
📄 Cargando 2024S-VBOG-056839.pdf
📄 Cargando 2024S-VBOG-056844.pdf
📄 Cargando 2024S-VBOG-056845.pdf
Esto es todo el contenido de `ml_papers:`

🆗 Todos los documentos estan cargados en ml_papers.
➖ Total de fragmentos: 31
➖ Los fragmentos son cada una de las hojas de cada uno de los 5 archivos en la carpeta ../../../assets/DG_docs/PDFs_test/
➖ Este script se ejecuta desde c:\Users\devel\UNAD\INVIAS\INVIAS_NLP\MMO_codes\langc\v01
➖ Este es el contenido de la última hoja cargada page_content='d. Las tasas de peajes serán diferenciales, es decir, se fijarán en proporción a las
distancias recorridas, las características vehiculares y sus respectivos costos de
operación. 
e. Para la determinación del valor del peaje y de las tasas de valorización, en las
vías nacionales, se tendrá en cuenta un criterio de equidad fiscal”.
4. Sírvase dar a conocer cuáles son los servicios que por normativa están
obligados a brindar las concesion

## 🔒 4. Split de documents

### 🔓

Solo ejecutar este script para desarrollar el proceso de embedding; este script depende del script anterior 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    # AI_Queries\code_explanation\ai_query-langc_v01-chunk_usage.md 
    # AI_Queries\code_explanation\ai_query-langc_v01-max_tokens_Chatgptmodels.md 
    # AI_Queries\code_explanation\ai_query-langc_v01-meaning_inputpromptandanswer.md 
    # AI_Queries\code_explanation\ai_query-langc_v01-retrieval_meaning.md
    
    chunk_overlap=200,
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [ ]:
len(documents), documents[0]

## 🔒 5. Embeddings e ingesta a base de datos vectorial 

⚠️ advertencia de uso de esta sección ⚠️

### 🔓 5.1.

Solo ejecutar este script para desarrolla el proceso de embedding; este script depende del script anterior

**Aquí se consume recurso de la API de OpenAI**

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# AI_Queries\code_explanation\ai_query-langc_v01-Embeddings_and_Vector_Store_Ingestion.md

# 1. Crear embeddings con el modelo oficial de OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 2. Definir carpeta para almacenar la base de datos vectorial
persist_directory = "chroma_db" #

# 3. Crear la base desde documentos y embeddings
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

# 4. Guardar la base en disco
vectorstore.persist()
print("✅ Base de datos Chroma guardada en:", persist_directory)


# 5. Cargar la base vectorial guardada en disco
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_db"
)

# 6. Usar como retriever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
    )

### 5.2.

Fue necesario crear copia de las lineas "*# 1. Crear embeddings con el modelo oficial de OpenAI*", "*# 5. Cargar la base vectorial guardada en disco*" y "*# 6. Usar como retriever*" para **solamente hacer uso de la Based de Datos de embeddings `chroma_db` ya creada y evitar recalcularla**.

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 1. Modelo de embeddings (debe ser el mismo usado al crear la base)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 5. Cargar la base vectorial guardada en disco
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_db"
)

# 6. Usar como retriever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

C:\Users\devel\AppData\Local\Temp\ipykernel_184\3507217838.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
C:\Users\devel\AppData\Local\Temp\ipykernel_184\3507217838.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


## 6. Modelos de chat y cadenas para consulta de información

In [3]:
#AI_Queries/code_explanation/ai_query-langc_v01-Chat_Models_and_Retrieval_Chains_for_Information_Querying.md

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key=api_key_environ,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

# AI_Queries/code_explanation/ai_query-langc_v01-RetrievalQA.from_chain_type_usage.md
qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

C:\Users\devel\AppData\Local\Temp\ipykernel_184\171208245.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


In [5]:
# AI_Queries/code_explanation/ai_query-langc_v01-qa_chain.run()_usage.md

query = "qué es CCPT?"
qa_chain.run(query)

'CCPT se refiere al Programa Caminos Comunitarios para la Paz Total. Es un programa orientado a la identificación de necesidades en la red vial regional del país, donde cualquier Organismo de Acción Comunal, comunidad étnica o entidad sin ánimo de lucro puede presentar sus necesidades para la planeación y estructuración de los tramos a intervenir, con la participación activa de la comunidad.'

In [ ]:
query = "qué hace complicado entrenar un modelo como el fingpt?"
qa_chain.run(query)

In [ ]:
query = "qué es fast segment?"
qa_chain.run(query)

In [ ]:
query = "cuál es la diferencia entre fast sam y mobile sam?"
qa_chain.run(query)

## 7. LangChain para INVIAS


### 7.1. Descripción del código

El archivo `langc_v01.ipynb` es un **notebook de Jupyter** diseñado para mostrar paso a paso cómo construir un **sistema de consulta de información basado en documentos PDF usando LangChain y modelos de lenguaje de OpenAI**. 

#### Propósito del archivo `langc_v01.ipynb`

Implementar un **chatbot inteligente** que pueda:

1. **Leer documentos PDF** institucionales.
2. **Convertirlos en embeddings semánticos** usando OpenAI.
3. **Almacenarlos en una base vectorial persistente** (Chroma).
4. **Responder preguntas** formuladas en lenguaje natural usando LLMs.

#### Estructura del notebook

1. **Carga y lectura de documentos PDF**

   * Usa loaders como `PyPDFLoader`.
   * Fragmenta el contenido en chunks con metadatos.
   * Prepara los datos para el embedding.

2. **Generación de embeddings y base vectorial**

   * Usa `OpenAIEmbeddings` (`text-embedding-ada-002`).
   * Crea y guarda una base en Chroma (`chroma_db`).
   * Permite que la búsqueda de contexto sea semántica, no solo por palabras clave.

3. **Creación del `retriever`**

   * Define cuántos documentos relevantes recuperar (`k=3` o `k=5`).
   * Es el núcleo del sistema RAG (Retrieval-Augmented Generation).

4. **Construcción de la cadena de QA (`RetrievalQA`)**

   * Conecta el `retriever` con un modelo de lenguaje (`ChatOpenAI`).
   * Puede configurarse para devolver solo la respuesta, o también las fuentes.

5. **Ejecución de consultas**

   * Envía preguntas como `"¿Qué es CCPT?"` o `"¿Qué acciones tomó el INVÍAS en El Tarrita?"`.
   * El sistema responde basándose en los documentos cargados.

> Este codigo viene de [langc_v01.ipynb](../../../Platzi_codes/langc/v01/langc_v01.ipynb). Este código desarrollo tiene [Code Explanation](../../../AI_Queries/code_explanation/).  

### 7.2. Hallazgos 🚩

1. No cita las fuentes de donde extrae la información.